# 如何进行工具/函数调用

:::info
我们互换使用工具调用和函数调用这两个术语。虽然函数调用有时指的是单个函数的调用，
但我们将所有模型都视为能够在每条消息中返回多个工具或函数调用。
:::

工具调用允许模型通过生成符合用户定义模式的输出来响应给定的提示。虽然名称暗示模型正在执行
某些操作，但实际上并非如此！模型只是提出工具的参数，而实际运行工具（或不运行）取决于用户 - 
例如，如果您想从非结构化文本中[提取符合某种模式的输出](/docs/tutorials/extraction)，
您可以给模型一个"提取"工具，该工具接受与所需模式匹配的参数，然后将生成的输出作为您的最终结果。

工具调用包括名称、参数字典和可选标识符。参数字典的结构为`{参数名: 参数值}`。

许多LLM提供商，包括[Anthropic](https://www.anthropic.com/)、
[Cohere](https://cohere.com/)、[Google](https://cloud.google.com/vertex-ai)、
[Mistral](https://mistral.ai/)、[OpenAI](https://openai.com/)等，
都支持工具调用功能的变体。这些功能通常允许向LLM的请求中包含可用工具及其模式，
并且响应中包含对这些工具的调用。例如，给定一个搜索引擎工具，LLM可能通过首先发出对搜索引擎的调用
来处理查询。调用LLM的系统可以接收工具调用，执行它，并将输出返回给LLM以指导其响应。
LangChain包含一套[内置工具](/docs/integrations/tools/)，并支持几种定义自己的[自定义工具](/docs/how_to/custom_tools)的方法。
工具调用对于构建[使用工具的链和代理](/docs/how_to#tools)以及更广泛地从模型获取结构化输出非常有用。

提供商采用不同的约定来格式化工具模式和工具调用。
例如，Anthropic在更大的内容块内返回解析的结构作为工具调用：
```python
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]
```
而OpenAI将工具调用分离为不同参数，参数为JSON字符串：
```python
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}
```
LangChain实现了用于定义工具、将工具传递给LLM以及表示工具调用的标准接口。

## 将工具传递给LLM

支持工具调用功能的聊天模型实现了`.bind_tools`方法，该方法接收LangChain [工具对象](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool)列表
并以其预期格式将它们绑定到聊天模型。之后对聊天模型的调用将在其对LLM的调用中包含工具模式。

例如，我们可以使用`@tool`装饰器在Python函数上定义自定义工具的模式：

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """将a和b相加。"""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """将a和b相乘。"""
    return a * b


tools = [add, multiply]

或者下面，我们使用Pydantic定义模式：


In [ ]:
from pydantic import BaseModel, Field


# 注意这里的文档字符串非常重要，它们将与类名一起传递给模型。
class Add(BaseModel):
    """将两个整数相加。"""

    a: int = Field(..., description="第一个整数")
    b: int = Field(..., description="第二个整数")


class Multiply(BaseModel):
    """将两个整数相乘。"""

    a: int = Field(..., description="第一个整数")
    b: int = Field(..., description="第二个整数")


tools = [Add, Multiply]

我们可以如下将它们绑定到聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  overrideParams={{fireworks: {model: "accounts/fireworks/models/firefunction-v1", kwargs: "temperature=0"}}}
/>

我们可以使用`bind_tools()`方法来处理将`Multiply`转换为"工具"
并将其绑定到模型（即，每次调用模型时都传递它）。

In [67]:
# | echo: false
# | output: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [68]:
llm_with_tools = llm.bind_tools(tools)

## 工具调用

如果LLM响应中包含工具调用，它们将作为[工具调用](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolCall.html#langchain_core.messages.tool.ToolCall)对象列表
附加到相应的[消息](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html#langchain_core.messages.ai.AIMessage)或
[消息块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk)的`.tool_calls`属性中。
`ToolCall`是一个类型化字典，包括工具名称、参数值字典和（可选的）标识符。没有工具调用的消息默认为此属性的空列表。

示例：

In [ ]:
query = "3 * 12等于多少？另外，11 + 49等于多少？"

llm_with_tools.invoke(query).tool_calls

[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_1Tdp5wUXbYQzpkBoagGXqUTo'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_k9v09vYioS3X0Qg35zESuUKI'}]

`.tool_calls`属性应该包含有效的工具调用。注意，有时候模型提供商可能会输出格式错误的工具调用（例如，不是有效JSON的参数）。
在这些情况下解析失败时，[InvalidToolCall](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.InvalidToolCall.html#langchain_core.messages.tool.InvalidToolCall)实例
将被填充到`.invalid_tool_calls`属性中。`InvalidToolCall`可以有名称、字符串参数、标识符和错误消息。

如果需要，[输出解析器](/docs/how_to#output-parsers)可以进一步处理输出。例如，我们可以转换回原始的Pydantic类：

In [16]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

[Multiply(a=3, b=12), Add(a=11, b=49)]

### 流式处理

当工具在流式上下文中被调用时，[消息块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk)
将通过`.tool_call_chunks`属性中的列表填充[工具调用块](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolCallChunk.html#langchain_core.messages.tool.ToolCallChunk)对象。
`ToolCallChunk`包括工具`name`、`args`和`id`的可选字符串字段，并包括可选的整数字段`index`，可用于将块连接在一起。这些字段是可选的，因为工具调用的部分可能会跨不同的块流式传输
（例如，包含参数子字符串的块可能对工具名称和ID具有空值）。

因为消息块继承自其父消息类，带有工具调用块的[AIMessageChunk](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessageChunk.html#langchain_core.messages.ai.AIMessageChunk)
也将包含`.tool_calls`和`.invalid_tool_calls`字段。这些字段是从消息的工具调用块中尽可能解析出来的。

请注意，目前并非所有提供商都支持工具调用的流式处理。

示例：

In [17]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_d39MsxKM5cmeGJOoYKdGBgzc', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'Add', 'args': '', 'id': 'call_QJpdxD9AehKbdXzMHxgDMMhs', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


请注意，添加消息块将合并它们对应的工具调用块。这是LangChain的各种[工具输出解析器](/docs/how_to/output_parser_structured)支持流式处理的原理。

例如，下面我们累积工具调用块：

In [18]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

[]
[{'name': 'Multiply', 'args': '', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a"', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, ', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '', 'id': 'call_tYHYdEV2YBvzDcSCiFCExNvw', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '{"a"', 'id': 'call_tYHYdEV2YBvzDcSCiFCExNvw', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_erKtz8z3e681cmxYKbRof0NS', 'index': 0}, {'name': 'Add', 'args': '{"a": 11,', 'id': 'call_

In [19]:
print(type(gathered.tool_call_chunks[0]["args"]))

<class 'str'>


下面我们累积工具调用以演示部分解析：

In [20]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

[]
[]
[{'name': 'Multiply', 'args': {}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_UjSHJKROSAw2BDc8cp9cSv4i'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BXqUtt6jYCwR1DguqpS2ehP0'}, 

In [21]:
print(type(gathered.tool_calls[0]["args"]))

<class 'dict'>


## 将工具输出传递给模型

如果我们使用模型生成的工具调用来实际调用工具，并想将工具结果传回模型，我们可以使用`ToolMessage`来实现。

In [117]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K5DsWEmgt6D08EI9AFu9NaL1', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_qywVrsplg0ZMv7LHYYMjyG81', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1a0b8cdd-9221-4d94-b2ed-5701f67ce9fe-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_K5DsWEmgt6D08EI9AFu9NaL1'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_qywVrsplg0ZMv7LHYYMjyG81'}]),
 ToolMessage(content='36', tool_call_id='call_K5DsWEmgt6D08EI9AFu9NaL1'),
 ToolMessage(content='60', tool_call_id='call_qywVrsplg0ZMv7LHYYMjyG81')]

In [118]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 is 36 and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 171, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6c8093c-b16a-4c92-8308-7c9ac998118c-0')

## 少样本提示

对于更复杂的工具使用，向提示中添加少样本示例非常有用。我们可以通过向提示中添加带有`ToolCall`的`AIMessage`和相应的`ToolMessage`来实现这一点。

例如，即使有一些特殊指令，我们的模型也可能因运算顺序而混淆：

In [ ]:
llm_with_tools.invoke(
    "119乘以8减去20是多少。不要自己做任何计算，只使用工具进行计算。遵守运算顺序"
).tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_Dl3FXRVkQCFW4sUNYOe4rFr7'},
 {'name': 'Add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_n03l4hmka7VZTCiP387Wud2C'}]

模型不应该尝试进行任何加法运算，因为它在技术上还不知道119 * 8的结果。

通过添加带有示例的提示，我们可以纠正这种行为：

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "317253和128472的乘积加上4是多少", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "317253和128472的乘积加上4等于16505054788",
        name="example_assistant",
    ),
]

system = """你不擅长数学，但你是使用计算器的专家。

使用过去的工具使用情况作为正确使用工具的示例。"""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("119乘以8减去20是多少").tool_calls

[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_MoSgwzIhPxhclfygkYaKIsGZ'}]

看起来这次我们获得了正确的输出。

这是[LangSmith跟踪](https://smith.langchain.com/public/f70550a1-585f-4c9d-a643-13148ab1616f/r)的样子。

## 下一步

- **输出解析**：查看[OpenAI工具输出解析器](/docs/how_to/output_parser_structured)
  了解如何将函数调用API响应提取为各种格式。
- **结构化输出链**：[一些模型有构造函数](/docs/how_to/structured_output)可以
  为您处理创建结构化输出链。
- **工具使用**：了解如何构建调用被调用工具的链和代理，请参阅
  [这些指南](/docs/how_to#tools)。